In [ ]:
%load_ext autoreload
%autoreload 2

# `src.abstract_syntax_tree`

In [ ]:
from src.abstract_syntax_tree import Node, AbstractSyntaxTree

## `Node`

In [ ]:
node_with_set_value = Node(id=1, kind="TEST", value=1)
print(node_with_set_value)

In [ ]:
node_with_default_value = Node(id=1, kind="TEST")
print(node_with_default_value)

## `AbstractSyntaxTree`

In [ ]:
tree = AbstractSyntaxTree()
print(tree.root)

# `src.syntax_parser`

In [ ]:
from src.syntax_parser import SyntaxParser

## `SyntaxParser`

In [ ]:
parser = SyntaxParser()
vars(parser)

### `term`

In [ ]:
test_id_term = parser.term("ID", 1)
print(test_id_term)

In [ ]:
# This should raise Exceptions, as the call is malformed.

try:
    test_failed_term = parser.term(
        symbol="OTHER",
        id=1,
        lhs=Node(-1, "TEST"),
        rhs=Node(-2, "TEST")
    )
    print(test_failed_term)
except UnboundLocalError as e:
    print(e)
    print("Test case 1: all parameters were set")

try:
    test_failed_term = parser.term(
        symbol=["OTHER"],
        id=1
    )
    print(test_failed_term)
except UnboundLocalError as e:
    print(e)
    print("Test case 2: `term` case, but symbol is a list")

try:
    test_failed_term = parser.term(
        symbol="OTHER",
        lhs=Node(-1, "TEST"),
        rhs=Node(-2, "TEST")
    )
    print(test_failed_term)
except UnboundLocalError as e:
    print(e)
    print("Test case 3: `parenthesis_expression` case, but symbol is a str")

In [ ]:
test_int_term = parser.term("INT", 1)
print(test_int_term)

In [ ]:
test_parenthesis_term = parser.term(
    symbol=["LPAR", "ID", "RPAR"],
    lhs=Node(-1, "TEST_LHS"),
    rhs=Node(-2, "TEST_RHS")
)
print(test_parenthesis_term)

for child in test_parenthesis_term.children:
    print(child)

### `parenthesis_expression`

In [ ]:
lhs = Node(id=-1, kind="INT", value=1)
rhs = Node(id=-2, kind="INT", value=2)

In [ ]:
# Generates a SET node

test_parenthesis_expression_set = parser.parenthesis_expression(
    symbol=["LPAR", "ID", "RPAR"],
    lhs=lhs,
    rhs=rhs
)
print(test_parenthesis_expression_set)

for child in test_parenthesis_expression_set.children:
    print(child)

In [ ]:
# Generates a LT node

test_parenthesis_expression_lt = parser.parenthesis_expression(
    symbol=["LPAR", "OTHER", "RPAR"],
    lhs=lhs,
    rhs=rhs
)
print(test_parenthesis_expression_lt)

for child in test_parenthesis_expression_lt.children:
    print(child)

In [ ]:
# Raises Exceptions, as the parenthesis are not correctly placed.

try:
    test_parenthesis_expression_placement = parser.parenthesis_expression(
        symbol=["RPAR", "OTHER", "LPAR"],
        lhs=lhs,
        rhs=rhs
    )
    print(test_parenthesis_expression_placement)
except Exception as e:
    print(e)
    print("Test case 1: the RPAR is before LPAR")

try:
    test_parenthesis_expression_missing = parser.parenthesis_expression(
        symbol=["RPAR", "OTHER"],
        lhs=lhs,
        rhs=rhs
    )
    print(test_parenthesis_expression_missing)
except Exception as e:
    print(e)
    print("Test case 2: missing LPAR")

try:
    test_parenthesis_expression_missing = parser.parenthesis_expression(
        symbol=["LPAR", "OTHER"],
        lhs=lhs,
        rhs=rhs
    )
    print(test_parenthesis_expression_missing)
except Exception as e:
    print(e)
    print("Test case 2: missing RPAR")

### `expression`

In [ ]:
lhs = Node(id=-1, kind="INT", value=1)
rhs = Node(id=-2, kind="INT", value=2)

In [ ]:
test_expression_lt = parser.expression(
    symbol="OTHER",
    lhs=lhs,
    rhs=rhs
)
print(test_expression_lt)

for child in test_expression_lt.children:
    print(child)

In [ ]:
test_expression_set = parser.expression(
    symbol="ID",
    lhs=lhs,
    rhs=rhs
)
print(test_expression_set)

for child in test_expression_set.children:
    print(child)

### `sum`

In [ ]:
lhs = Node(id=-1, kind="INT", value=1)
rhs = Node(id=-2, kind="INT", value=2)

In [ ]:
test_sum_plus = parser.sum(symbol="PLUS", lhs=lhs, rhs=rhs)
print(test_sum_plus)

for child in test_sum_plus.children:
    print(child)
    print(child.parent)

In [ ]:
test_sum_minus = parser.sum(symbol="MINUS", lhs=lhs, rhs=rhs)
print(test_sum_minus)

for child in test_sum_minus.children:
    print(child)
    print(child.parent)

### `comparison`

In [ ]:
lhs = Node(id=-1, kind="INT", value=1)
rhs = Node(id=-2, kind="INT", value=2)

In [ ]:
test_comparison = parser.comparison(lhs=lhs, rhs=rhs)
print(test_comparison)

for child in test_comparison.children:
    print(child)
    print(child.parent)

### `if_statement`

In [ ]:
_parenthesis_expression = Node(id=1, kind="PAR")
_statement = Node(id=2, kind="STTMNT")

In [ ]:
if_statement = parser.if_statement(_parenthesis_expression, _statement)
print(if_statement)

for child in if_statement.children:
    print(child)

### `if_else_statement`

In [ ]:
_parenthesis_expression = Node(id=1, kind="PAR")
_statement_if = Node(id=2, kind="STTMNT")
_statement_else = Node(id=3, kind="STTMNT")

In [ ]:
if_else_statement = parser.if_else_statement(_parenthesis_expression, _statement_if, _statement_else)
print(if_else_statement)

for child in if_else_statement.children:
    print(child)

### `while_statement`

In [ ]:
_parenthesis_expression = Node(id=1, kind="PAR")
_statement = Node(id=2, kind="STTMNT")

In [ ]:
while_statement = parser.while_statement(_parenthesis_expression, _statement)
print(while_statement)

for child in while_statement.children:
    print(child)

### `do_while_statement`

In [ ]:
_parenthesis_expression = Node(id=1, kind="PAR")
_statement = Node(id=2, kind="STTMNT")

In [ ]:
do_while_statement = parser.do_while_statement(_parenthesis_expression, _statement)
print(do_while_statement)

for child in do_while_statement.children:
    print(child)

### `program`

In [ ]:
_statement = Node(id=1, kind="STTMNT")

In [ ]:
program = parser.program(_statement)
print(program)

for child in program.children:
    print(child)

# `CodeGenerator`

In [ ]:
from src.code_generator import CodeGenerator

## `generate_code`

In [ ]:
generator = CodeGenerator()

### `simple_node`

`VAR`, `CST`, `ADD`, `SUB`, `LT`.

In [ ]:
# VAR
var_node = Node(id=1, kind="VAR", value=23)
generator.generate_code(var_node)
print(generator)

In [ ]:
# CST
cst_node = Node(id=2, kind="CST", value=23)
generator.generate_code(cst_node)
print(generator)

In [ ]:
# ADD
lhs = Node(id=-1, kind="CST", value=1)
rhs = Node(id=-2, kind="VAR", value=2)
add_node = parser.sum(symbol="PLUS", lhs=lhs, rhs=rhs)

generator.generate_code(add_node)
print(generator)

In [ ]:
# SUB
lhs = Node(id=-3, kind="VAR", value=11)
rhs = Node(id=-4, kind="CST", value=22)
sub_node = parser.sum(symbol="MINUS", lhs=lhs, rhs=rhs)

generator.generate_code(sub_node)
print(generator)

In [ ]:
# LT
lhs = Node(id=-1, kind="CST", value=1)
rhs = Node(id=-2, kind="CST", value=2)
lt_node = parser.comparison(lhs=lhs, rhs=rhs)

generator.generate_code(lt_node)
print(generator)

### `parse_set_node`

In [ ]:
lhs = Node(id=-1, kind="CST", value=1)
rhs = Node(id=-2, kind="CST", value=2)
set_node = parser.expression(
    symbol="ID",
    lhs=lhs,
    rhs=rhs
)

generator.generate_code(set_node)
print(generator)